# MIMIC Notes Pre-Processing

Pre-processing MIMIC notes for language model and CUIs

## Imports and Inits

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import re
import random
import datetime

from pathlib import Path

In [2]:
PATH = Path('data')

## Functions

## Grab sample data from MIMIC

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 2000

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

In [4]:
# limit = 50
# queries = [
#     f"""
#     select category, text from correctnotes where category=\'{cats.iloc[7]['category']}\' order by random() limit {limit}
#     """
# ]

In [5]:
%%time
dfs = []

con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)
# df.set_index('row_id', inplace=True)
print(df.shape)

(23152, 2)
CPU times: user 103 ms, sys: 77.8 ms, total: 181 ms
Wall time: 19.4 s


1. Get list of redacted types using `re`
2. replace it with appropriate holder tokens

Below is a list of redacted items with an example and the replacement token.

Redacted items:
* [x] First Name: `[**First Name (Titles) 137**]`, `t_firstname`
* [x] Last Name: `[**Last Name (Titles) **]`, `t_lastname`
* [x] Initials: `[**Initials (NamePattern4) **]`, `t_initials`
* [x] Name: `[**Name (NI) **]`, `t_name`
* [x] Doctor First Name: `[**Doctor First Name 1266**]`, `t_doctor_firstname`
* [x] Doctor Last Name: `[**Doctor Last Name 1266**]`, `t_doctor_lastname`
* [x] Known Last Name: `[**Known lastname 658**]`, `t_lastname`
* [x] Hospital: `[**Hospital1 **]`, `t_hospital`
* [x] Hospital Unit Name: `**Hospital Unit Name 10**`, `t_hospital`
* [x] Company: `[**Company 12924**]`, `t_workplace`
* [x] University/College: `[**University/College **]`, `t_workplace`
* [x] Date of format YYYY-M-DD: `[**2112-4-18**]`, `t_fulldate`
* [x] Year: `[**Year (4 digits) **]`, `t_year`
* [x] Year YYYY format: `[**2119**]`, `t_year` - I use a regex `\b\d{4}\b` that will match **any** 4 digits which might be problematic, but for the most part 4 digits by itself seems to indicate a year.
* [x] Date of format M-DD: `[**6-12**]`, `[**12/2151**]`, `t_monthday`
* [x] Month/Day: `[**Month/Day (2) 509**]`, `t_monthday`
* [x] Month (only): `[**Month (only) 51**]`, `t_month`
* [x] Holiday: `[**Holiday 3470**]`, `t_month`
* [x] Date Range: `[**Date range (1) 7610**]`, `t_daterange`
* [x] Country: `[**Country 9958**]`, `t_country`
* [x] State: `[**State 3283**]`, `t_state`
* [x] Location: `**Location (un) 2432**`, `t_location`
* [x] Telephone/Fax: `[**Telephone/Fax (3) 8049**]`, `t_phone`
* [x] Clip Number: `[**Clip Number (Radiology) 29923**]`, `t_radclip_id`
* [x] Pager Numeric Identifier: `[**Numeric Identifier 6403**]`, `t_pager_id`
* [x] Pager Number: `[**Pager number 13866**]`, `t_pager_id`
* [x] Social Security Number: `[**Security Number 10198**]`, `t_ssn`
* [x] Serial Number: `[**Serial Number 3567**]`, `t_sn`
* [x] Medical Record Number: `[**Medical Record Number **]`, `t_mrn`
* [x] Provider Number: `[**Provider Number 12521**]`, `t_provider_no`
* [x] Age over 90: `[**Age over 90 **]`, `t_oldage`
* [x] Time: `12:52 PM`, split into 6 segments by the hour and replace with the following tokens: `midnight, dawn, forenoon, afternoon, dusk, night`
* Just numbers: `[** 7901**]`
* Wardname
* Pharmacy MD Number* 

In [6]:
pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)

In [7]:
def redacorator(func):
    def replace(match):
        ori = match.group()
        text = match.group().lower()
        if set(ori) == set(' *]['):
            ori = ''
        return func(text, ori)
    return replace

@redacorator
def replace_name(text, ori):
    r = ori
    if 'name' in text:
        r = 't_name'
        if 'last' in text:
            if 'doctor' in text:
                r = 't_doctor_lastname'
            else:
                r = 't_lastname'
        elif 'first' in text:
            if 'doctor' in text:
                r = 't_doctor_firstname'
            else:
                r = 't_firstname'
        elif 'initials' in text:
            r = 't_initials'
    return r

@redacorator
def replace_place(text, ori):
    r = ori
    if 'hospital' in text:
        r = 't_hospital'
    elif ('company' in text) or ('university/college' in text):
        r = 't_workplace'
    elif 'location' in text:
        r = 't_location'
    elif 'country' in text:
        r = 't_country'
    elif 'state' in text:
        r = 't_state'
    elif ('address' in text) or ('po box' in text):
        r = 't_address'
    return r

@redacorator
def replace_dates(text, ori):
    r = ori
    if 'year' in text or re.search(r'\b\d{4}\b', text):
        r = 't_year'
    elif re.search(r'\d{4}-\d{0,2}-\d{0,2}', text):
        r = 't_fulldate'        
    elif (re.search(r'\d{0,2}-\d{0,2}', text)) or (re.search(r'\d{0,2}\/\d{0,2}', text)) or ('month/day' in text):
        r = 't_monthday'
    elif 'month' in text:
        r = 't_month'
    elif 'holiday' in text:
        r = 't_holiday'
    elif 'date range' in text:
        r = 't_daterange'
    return r

@redacorator
def replace_identifiers(text, ori):
    r = ori
    if ('numeric identifier' in text) or ('pager number' in text):
        r = 't_pager_id'
    elif '(radiology)' in text:
        r = 't_radclip_id'
    elif 'social security number' in text:
        r = 't_ssn'
    elif 'medical record number' in text:
        r = 't_mrn'
    elif 'age over 90' in text:
        r = 't_oldage'
    elif 'serial number' in text:
        r = 't_sn'
    elif 'unit number' in text:
        r = 't_unit_no'
    elif 'md number' in text:
        r = 't_md_no'
    elif 'telephone/fax' in text:
        r = 't_phone'
    elif 'provider number' in text:
        r = 't_provider_no'
    return r

def replace_redacted(text):    
    pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)
    
    # replace name types
    text = pat.sub(replace_name, text)
    
    # replace place types
    text = pat.sub(replace_place, text)
    
    # replace date types
    text = pat.sub(replace_dates, text)

    # replace person identifier types
    text = pat.sub(replace_identifiers, text)
    
#     text = re.sub(r'\[\*\*(\d{2})\*\*\] \b[A|P].?M.?\b', 't_hour', text, re.IGNORECASE)      
    return text

In [82]:
def replace_time(match):
    try:
        time = match.group().strip().lower()

        f, s = t.split()
        s = 'am' if s[0] == 'a' else 'pm'
        l, r = f.split(':')
        if l == '':
            if r == '':
                r = str(0).zfill(2)
            l = str(12)
        if int(l) > 12:
            l = str(int(l) % 12)
        f = ':'.join([l, r])
        time = ' '.join([f, s])
        
        d = datetime.datetime.strptime(time, '%I:%M %p')
        if d.hour >= 0 and d.hour < 4:
            time = 't_midnight'
        elif d.hour >= 4 and d.hour < 8:
            time = 't_dawn'
        elif d.hour >= 8 and d.hour < 12:
            time = 't_forenoon'
        elif d.hour >= 12 and d.hour < 16:
            time = 't_afternoon'
        elif d.hour >=16 and d.hour <20:
            time = 't_dusk'
        else:
            time = 't_night'
    except ValueError:
        time = match.group()
    return time

def misc_scrub(text):
    # replace different types of "year old" with year_old
    # matches: y.o., y/o, years old. year old, yearold
    text = re.sub(r'\byears? ?old\b|\by(?:o|r)*[ ./-]*o(?:ld)?\b', 't_year_old',
               text, flags=re.IGNORECASE)
    
    # replaces yr, yr's, yrs with years
    text = re.sub(r'\byr[\'s]*\b', 'years', text, re.IGNORECASE)
    
    # replace Pt and pt with patient, and IN/OUT/OT PT with patient
    # Note: PT also refers to physical therapy and physical therapist
    text = re.sub(r'\b[P|p]t.?|\b(IN|OU?T) PT\b', 'patient', text)
    
    text = re.sub(r'\d{0,2}:\d{0,2} \b[A|P]\.?M\.?\b', replace_time, text, flags=re.IGNORECASE)
    return text

In [9]:
def scrub_text(text):
    # replace redacted info with tokens
    text = replace_redacted(text)
    
    # misc scrubbing
    text = misc_scrub(text)
    
    return text

In [80]:
times = ['14:45 PM', ':00 AM', '40:00 PM', '13:00 pm', '21:30 PM', ': AM', '9:00 p.m', '5:00 a.m',\
         '4:11 A.M', '4:45 a.m', '6:56 P.M', ': A.M', '19:06 p.m']
times = list(map(str.lower, times))

In [81]:
for t in times:
    f, s = t.split()
    s = 'am' if s[0] == 'a' else 'pm'
    l, r = f.split(':')
    if l == '':
        if r == '':
            r = str(0).zfill(2)
        l = str(12)
    if int(l) > 12:
        l = str(int(l) % 12)
    f = ':'.join([l, r])
    nt = ' '.join([f, s])
    
    d = datetime.datetime.strptime(nt, '%I:%M %p')
    if d.hour >= 0 and d.hour < 4:
        time = 't_midnight'
    elif d.hour >= 4 and d.hour < 8:
        time = 't_dawn'
    elif d.hour >= 8 and d.hour < 12:
        time = 't_forenoon'
    elif d.hour >= 12 and d.hour < 16:
        time = 't_afternoon'
    elif d.hour >=16 and d.hour <20:
        time = 't_dusk'
    else:
        time = 't_night'

    print(time)


t_afternoon
t_midnight
t_dusk
t_afternoon
t_night
t_midnight
t_night
t_dawn
t_dawn
t_dawn
t_dusk
t_midnight
t_dusk


In [70]:
str(0).zfill(2)

'00'

In [44]:
t = times[0]

In [55]:
a,b = 2,2

In [45]:
f, s = t.split()
l, _ = f.split(':')

In [51]:
l = int(l)

In [12]:
pat2 = re.compile(r'\d{0,2}:\d{0,2} \b[A|P]\.?M\.?\b', re.IGNORECASE)

In [ ]:
test = df.iloc[959]['text']
# for m in pat.finditer(test):
#     print(m)

print(test)

In [ ]:
out = scrub_text(test)
print(out)

In [ ]:
for i, row in df.iterrows():
    if len(pat2.findall(row['text'])) != 0:
        print(i, pat2.findall(row['text']))

In [84]:
for i, row in df.iterrows():
    if len(pat2.findall(row['scrubbed'])) != 0:
        print(i, pat2.findall(row['scrubbed']))

In [ ]:
for m in pat2.finditer(test):
    print(m)

In [ ]:
out = scrub_text(test)
# for m in pat.finditer(out):
#     print(m)

In [ ]:
print(out)

In [83]:
df['scrubbed'] = df['text'].apply(scrub_text)

In [ ]:
for i, row in df.iterrows():
    if len(pat.findall(row['scrubbed'])) != 0:
        print(i, pat.findall(row['scrubbed']))